## Importação das Bibiliotecas

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from sqlalchemy import create_engine
import pyodbc

## Criando uma SparkSession

In [2]:
spark = SparkSession.builder.appName("LerDadosEnem").getOrCreate()


## Processo de Leitura do CSV de Participantes


In [3]:
#Path dos arquivos

caminho_arquivo_2022 = r'E:\Estudos\SQL\Datasets\ENEM\2022\microdados_enem_2022\DADOS\MICRODADOS_ENEM_2022.csv'
caminho_arquivo_2021 = r'E:\Estudos\SQL\Datasets\ENEM\2021\microdados_enem_2021\DADOS\MICRODADOS_ENEM_2021.csv'
caminho_arquivo_2020 = r'E:\Estudos\SQL\Datasets\ENEM\2020\microdados_enem_2020\DADOS\MICRODADOS_ENEM_2020.csv'
caminho_arquivo_2019 = r'E:\Estudos\SQL\Datasets\ENEM\2019\microdados_enem_2019\DADOS\MICRODADOS_ENEM_2019.csv'

In [67]:
#Função Paa ler os arquivos CSV
def lerCSV(spark, caminho_arquivo):
    df = spark.read.csv(caminho_arquivo, sep=';', header=True, inferSchema=True, encoding='ISO-8859-1')
    colunas_selecionadas = df.columns[12:19]
    return df.select(colunas_selecionadas)


In [69]:
##Chama função quer ler o arquivo CSV e Transforma para DF.SPARK

dfEscola22 = lerCSV(spark, caminho_arquivo_2022)
dfEscola21 = lerCSV(spark, caminho_arquivo_2021)
dfEscola20 = lerCSV(spark, caminho_arquivo_2020)
dfEscola19 = lerCSV(spark, caminho_arquivo_2019)


In [70]:
#Unir os DF em um só 

dfDadosGerais = dfEscola22.union(dfEscola21).union(dfEscola20).union(dfEscola19)


In [76]:
dfPreencher = dfDadosGerais.fillna(-2)  # Preenche com -2(sem ref), por exemplo


In [88]:
# Realize o agrupamento no DataFrame original
dfPreencherDistinct = dfPreencher.select(
    "CO_MUNICIPIO_ESC",
    "NO_MUNICIPIO_ESC",
    "CO_UF_ESC",
    "SG_UF_ESC",
    "TP_DEPENDENCIA_ADM_ESC",
    "TP_LOCALIZACAO_ESC",
    "TP_SIT_FUNC_ESC"
).distinct()


# Salvar em SQL 
Salvar os dados em uma tabela SQL


## Parametrizando a conexão do Banco SQL SERVER Usando JDBC

In [12]:
server_name = "jdbc:sqlserver://NOTEBOOK-LEO:1433"
database_name = "Enem"
table_name = "Escola"
username = "bi_leonardobo"
password = "0000"
url = server_name + ";" + "databaseName=" + database_name + ";"
print(url)


jdbc:sqlserver://NOTEBOOK-LEO:1433;databaseName=Enem;


In [13]:
# Nome da nova tabela que será criada
nomeTabela = table_name

In [15]:
# Configurações do banco de dados em ODBC para dropar
server_name = "NOTEBOOK-LEO\\SQLEXPRESS"
 
# Conecte-se ao banco de dados usando pyodbc
conn = pyodbc.connect(f"Driver={{SQL Server}};Server={server_name};Database={database_name};UID={username};PWD={password}")

# Crie um cursor
cursor = conn.cursor()

# Execute a operação DROP TABLE na tabela 'Escola'
truncate_query = f"DROP TABLE {table_name}"
cursor.execute(truncate_query)

# Commit as alterações no banco de dados
conn.commit()

In [90]:
# Escreva os novos dados em 'Escola' usando a conexão JDBC
dfPreencherDistinct.write \
        .format("jdbc") \
        .option("url", url) \
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .option("ssl", "true")\
        .option("trustServerCertificate", "true")\
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .option("SaveMode", "overWrite").save()
  


In [91]:
print(f'Tabela {nomeTabela} criada com sucesso no banco de dados.')


Tabela Provas criada com sucesso no banco de dados.
